In [13]:
import pandas as pd 
import numpy as np
import time
import math
import random
import os
from collections import Counter
import copy
import pickle
import os 
import csv
import re

In [14]:
# namedataset="Helpdesk"
# namedataset="BPI_Challenge_2012_W_Complete"
# namedataset="BPI_Challenge_2012_W"
# namedataset="BPI_Challenge_2012_A"
# namedataset="BPI_Challenge_2012_O"

In [15]:
for step in range(5):
    train_val_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/train_val_"+namedataset+".csv"
    train_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/train_"+namedataset+".csv"
    val_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/val_"+namedataset+".csv"
    test_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/test_"+namedataset+".csv"
    df_test=pd.read_csv(test_file)
    try:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_train=pd.read_csv(train_val_file)
    try:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp"]
    df_train.fillna(1, inplace=True)
    df_test.fillna(1,inplace=True)
    # df_valid.fillna(1,inplace=True)

    cont_trace = df_train['CaseID'].value_counts(dropna=False)
    cont_test_trace = df_test['CaseID'].value_counts(dropna=False)
    max_trace = max(cont_trace)
    max_test_trace = max(cont_test_trace)
    # max_valid_trace = max(cont_valid_trace)
    # real_max=max(max_trace,max_test_trace,max_valid_trace)
    real_max=max(max_trace,max_test_trace)

    test_listOfResource=df_test["Resource"].unique().tolist()
    train_listOfResource=df_train["Resource"].unique().tolist()
    # valid_listOfResource=df_valid["Resource"].unique().tolist()

    test_resourceset=set(test_listOfResource)
    train_resourceset=set(train_listOfResource)
    # valid_resourceset=set(valid_listOfaResource)

    # all_resource=list(valid_resourceset.union(test_resourceset.union(train_resourceset)))
    all_resource=list(test_resourceset.union(train_resourceset))

    all_resource.sort()

    listOfresourcesInt = list(range(0, len(all_resource)))
    resourceMapping=dict(zip(all_resource,listOfresourcesInt))
    # 统一活动值

    test_listOfEvents=df_test["Activity"].unique().tolist()
    train_listOfEvents=df_train["Activity"].unique().tolist()
    # valid_listOfEvents=df_valid["Activity"].unique().tolist()

    test_eventset=set(test_listOfEvents)
    train_eventset=set(train_listOfEvents)
    # valid_eventset=set(valid_listOfEvents)

    # all_events=list(valid_eventset.union(test_eventset.union(train_eventset)))
    all_events=list(test_eventset.union(train_eventset))

    all_events.sort()
    listOfeventsInt = list(range(0, len(all_events)))
    mapping = dict(zip(all_events, listOfeventsInt))

    df_test.Activity = [mapping[item] for item in df_test.Activity]
    # 将活动名称和id对应
    df_train.Activity = [mapping[item] for item in df_train.Activity]
    # 将活动名称和id对应

    df_test.Resource = [resourceMapping[item] for item in df_test.Resource]
    df_train.Resource = [resourceMapping[item] for item in df_train.Resource]

    # group by activity, resource and timestamp by caseid
    act = df_train.groupby('CaseID', sort=False).agg({'Activity': lambda x: list(x)})
    res = df_train.groupby('CaseID', sort=False).agg({'Resource': lambda x: list(x)})
    temp = df_train.groupby('CaseID', sort=False).agg({'Timestamp': lambda x: list(x)})
    # amount = df_train.groupby('CaseID', sort=False).agg({'Amount': lambda x: list(x)})

    from collections import Counter
    act,res,temp=act.loc[:,"Activity"].values,res.loc[:,"Resource"].values,temp.loc[:,"Timestamp"].values
    res_counter={resource:0 for resource in listOfresourcesInt}
    for res_trace in res:
        for resource in res_trace:
            try:
                res_counter[resource]+=1
            except:
                print("what")
    normal_res=[]
    abnormal_res=[]
    for key in res_counter:
        if res_counter[key]>=5:
            normal_res.append(key)
        else:
            abnormal_res.append(key)
            
    rg=re.compile("\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")
    def time_format(ttime):
        ttime=rg.search(ttime).group(0)
        try:
            date_format_str = '%Y/%m/%d %H:%M:%S.%f'
            conversion = time.mktime(time.strptime(ttime, date_format_str))
        except:
            date_format_str = '%Y-%m-%d %H:%M:%S'
            conversion = time.mktime(time.strptime(ttime, date_format_str))
        return conversion

    for i,times in enumerate(temp):
        temp[i]=[ time_format(ttime) for ttime in times]

    time_difference_lists=[]
    for times in temp:
        a=np.array(times)[1:]
        b=a-np.array(times)[:-1]
        time_difference_lists.append(np.concatenate((np.array([0]),b)))

    total_time_difference=[]
    for difference_list in time_difference_lists:
        total_time_difference+= difference_list.tolist()

    total_time_difference.sort()

    orgdifferencelist=total_time_difference[len(act):]

    Q1=orgdifferencelist[int(len(orgdifferencelist)+1)//4]
    Q2=orgdifferencelist[int(2*len(orgdifferencelist)+1)//4]
    Q3=orgdifferencelist[int(3*len(orgdifferencelist)+1)//4]
    interval=2*(Q3-Q1)/(len(orgdifferencelist)**(1/3))
    maxInterval=math.ceil((Q3+(Q3-Q1)*1.5)/interval)
    def gen_new_pattern(pattern_dict,index_list,father_pattern,database,min_sup,min_interval):
        # father_pattern 字符串格式的字符数组
        # father_pattern=[int(num) for num in father_pattern[1:-1].split(",")]
        # father_pattern = father_pattern[1:-1].split(",")
        
        length=len(father_pattern)
        temp_dict={}
        for pair in index_list:
            trace_index=pair[0]
            pattern_index=pair[1]
            mother_trace=database[trace_index]
            for i in range(1,min_interval+1):
                event_index=pattern_index+i
                if event_index>=len(mother_trace):
                    continue
                act=mother_trace[event_index]
                if act not in temp_dict:
                    temp_dict[act]=[[trace_index,event_index]]
                else:
                    temp_dict[act].append([trace_index,event_index])
        for k,v in temp_dict.items():
            if len(v)>=min_sup:
                new_pattern=father_pattern+","+k
                pattern_dict[new_pattern]=v
                gen_new_pattern(pattern_dict,v,new_pattern,database,min_sup,min_interval)
            
    def gen_one_dict(database,min_sup):
        tp_dict={}
        for i,trace in enumerate(database):
            for j,item in enumerate(trace):
                # tp_dict[str([item])]=tp_dict.get(str([item]),[]).append([i,j])
                if item not in tp_dict:
                    tp_dict[item]=[[i,j]]
                else:
                    tp_dict[item].append([i,j])
        count_dict={}

        for key,v in tp_dict.items():
            if len(v)>=min_sup:
                count_dict[key]=v

        return count_dict
            
    def res_clustering():
        res_act_list=[[0 for j in range(len(all_events))] for i in range(len(all_resource))]
        abnormal_res_act_list=[0 for j in range(len(all_events))]
        for act_trace,res_trace in zip(act,res):
            for activity,resource in zip(act_trace,res_trace):
                if resource in normal_res:
                    res_act_list[resource][activity]+=1
                else:
                    abnormal_res_act_list[activity]+=1
        for abn_res in abnormal_res:
            res_act_list[abn_res]=copy.deepcopy(abnormal_res_act_list)
            
        for i in range(len(res_act_list)):
            tsum=sum(res_act_list[i])
            for j in range(len(res_act_list[i])):
                if tsum!=0:
                    res_act_list[i][j]=(res_act_list[i][j])/(tsum)
        res_act_array=np.array(res_act_list)
        def cosine_similarity(a,b):
            return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))
        Similarity_matrix=[[0 for j in range(len(all_resource))] for i in range(len(all_resource))]
        for i in range(len(all_resource)):
            for j in range(len(all_resource)):
                Similarity_matrix[i][j]=cosine_similarity(res_act_array[i],res_act_array[j])
        Similarity_matrix
        from sklearn.cluster import SpectralClustering
        clustering = SpectralClustering().fit_predict(Similarity_matrix)
        return clustering
    res_cluster_label=res_clustering()
    res_cluster_log=[[res_cluster_label[resource] for resource in res_trace] for res_trace in res]
    def Discrete_continuous_properties(act,properties,interval_size,max_interval):
        act_prop_data=[]
        for act_trace,con_trace in zip(act,properties):
            temp_trace=[]
            for activity,amount in zip(act_trace,con_trace):
                dis_amount=str(math.floor(amount/interval_size) if (amount/interval_size) < max_interval else max_interval)
                item=str(activity)+";"+dis_amount
                temp_trace.append(item)
            act_prop_data.append(temp_trace)
        return act_prop_data
    def Bound_discrete_properties(act,properties):
        act_prop_data=[]
        for act_trace,dis_trace in zip(act,properties):
            temp_trace=[]
            for activity,item in zip(act_trace,dis_trace):
                pair=str(activity)+";"+str(item)
                temp_trace.append(pair)
            act_prop_data.append(temp_trace)
        return act_prop_data
    def generate_act_pattern(min_sup):
        act_data=[]
        for trace in act:
            act_data.append([str(act) for act in trace])
        pattern_dict_one=gen_one_dict(act_data,min_sup)
        min_interval=1
        pattern_dict={}
        for key,value in pattern_dict_one.items():
            gen_new_pattern(pattern_dict,value,key,act_data,min_sup,min_interval)
        return pattern_dict
    def generate_act_prop_pattern(act,props,min_sup,mode,interval_size,max_interval):
        if mode:
            act_prop_data=Discrete_continuous_properties(act,props,interval_size,max_interval)
        else:
            act_prop_data=Bound_discrete_properties(act,props)
        # print("props bound done")
        pattern_act_props_one=gen_one_dict(act_prop_data,min_sup)
        # print("one dict generatered")
        min_interval=1
        pattern_dict={}
        for key,value in pattern_act_props_one.items():
            gen_new_pattern(pattern_dict,value,key,act_prop_data,min_sup,min_interval)
        return pattern_dict

    def pattern_mining(alpha):
        min_sup=len(act)*alpha
        start=time.time()
        pattern_act_dict=generate_act_pattern(min_sup)
        pattern_act_time_dict=generate_act_prop_pattern(act,time_difference_lists,min_sup,1,interval,maxInterval)
        pattern_act_res_dict=generate_act_prop_pattern(act,res_cluster_log,min_sup,0,0,0)
        end=time.time()
        pickle.dump(pattern_act_dict,open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/pattern_act_dict_"+str(alpha)[:3]+".pkl","wb"))
        pickle.dump(pattern_act_time_dict,open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/pattern_act_time_dict_"+str(alpha)[:3]+".pkl","wb"))
        pickle.dump(pattern_act_res_dict,open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/pattern_act_res_dict_"+str(alpha)[:3]+".pkl","wb"))
        pickle.dump(res_cluster_label,open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/res_cluster_label_"+str(alpha)[:3]+".pkl","wb"))
        pickle.dump(maxInterval,open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/maxInterval_"+str(alpha)[:3]+".pkl","wb"))
        
        # print("generate time ",alpha,"  ",end-start)
        # print("act pattern length ",len(pattern_act_dict))
        # print("act res pattern length ",len(pattern_act_res_dict))
        # print("act time pattern length ",len(pattern_act_time_dict))
        return end-start,len(pattern_act_dict),len(pattern_act_res_dict),len(pattern_act_time_dict)

    with open("datasets_results/"+namedataset+"/fold"+str(step)+"/rq4.csv","w") as f:
        writer=csv.writer(f)
        writer.writerow(["",0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
        times=[]
        l1s=[]
        l2s=[]
        l3s=[]
        for i in range(1,11):
            alpha=0.1*i
            time1,l1,l2,l3=pattern_mining(alpha)
            times.append(time1)
            l1s.append(l1)
            l2s.append(l2)
            l3s.append(l3)
        row=["time"]+times
        writer.writerow(row)
        row=["pattern1"]+l1s
        writer.writerow(row)
        row=["pattern2"]+l2s
        writer.writerow(row)
        row=["pattern3"]+l3s
        writer.writerow(row)

        


/home/hsc/miniconda3/envs/py3.9/lib/python3.9/site-packages/sklearn/cluster/_spectral.py:658: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/home/hsc/miniconda3/envs/py3.9/lib/python3.9/site-packages/sklearn/cluster/_spectral.py:658: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/home/hsc/miniconda3/envs/py3.9/lib/python3.9/site-packages/sklearn/cluster/_spectral.py:658: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/home/hsc/miniconda3/envs/py3.9/lib/python3.9/site-packages/sklearn/cluster/_spectral.py:658: UserWarning: The spectral clustering API has changed. ``fit``now c